In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_coefficients( File ) :
    with open(File, 'r') as model :
        for line in model :
            performance = line.split(',')[0]
            coefficients = line.split(',')[1:]
    coefficients = [float(i) for i in coefficients]
    return coefficients

In [3]:
svm_coefficients_01 = get_coefficients( '../../Model_Building/MODEL/SVM_11.txt' )

In [4]:
type( svm_coefficients_01[0] )

float

In [5]:
# svm_coefficients_01

In [6]:
from numpy import array
# t0 = array( [1.] + test0.iloc[[0]].values.tolist()[0] )
# t1 = array( [1.] + test0.iloc[[1]].values.tolist()[0] )
# s0 = t0 - t1

In [7]:
# t0.shape

In [8]:
def score(coefficients, answer0, answer1):
    sub = answer0 - answer1
    score = sub.dot(coefficients)
    if score >= 0:
        return 1
    else :
        return -1

In [9]:
import operator
import collections, math
from collections import defaultdict
# A_0_score = 0
# print test0.icol[[30]]

def Rank_List( AnswerList, coefficients ):
    Score_Map = collections.defaultdict(lambda : 0)
    # print Score_Map
    # Score_Map = {}
    # Score_Map = defaultdict(lambda:0, Score_Map)
    for i in range( len(AnswerList) - 1 ):

        for j in range(i+1, len(AnswerList) ):

            #print (i, j)

            answer_i = array( [1.] + AnswerList.iloc[[i]].values.tolist()[0] )
            answer_j = array( [1.] + AnswerList.iloc[[j]].values.tolist()[0] )

            key_i = 'A_' + str(i) + '_score'
            key_j = 'A_' + str(j) + '_score'

            value_i = 0
            value_j = 0
            value_i += score(coefficients, answer_i, answer_j) 
            value_j -= score(coefficients, answer_i, answer_j)

            Score_Map[key_i] += value_i
            Score_Map[key_j] += value_j
    
    return Score_Map
    # print Score_Map
    # Sorted_scores = sorted(Score_Map.items(), key=operator.itemgetter(1), reverse=True)
    # print Sorted_scores
    # return Sorted_scores
    

In [10]:
def writeDict(Dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( Dict.items() ):
            f.write( str(key) + sep + str(value) + '\n' )

def writeList(List, filename) :
    with open(filename, 'w') as f:
        for i in range( len(List) ) :
            # print List[i][0]
            # print List[i][1]
            f.write( str(List[i][0]) + ':' + str(List[i][1]) + '\n')
            
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = int(values[1])
            #value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return dict 

In [11]:
import os 
data_path = './SVM_Data/TEST/'
file_list = os.listdir(data_path)

In [12]:
File_List_Len = len ( file_list )

In [13]:
File_List_Len

100

In [14]:
# test0 = pd.read_csv('./SVM_Data/TEST/test_0.csv')

In [15]:
# Answer_Rank = []
for i in range(File_List_Len) :
    input_file_name = 'test_' + str(i) + '.csv'
    # print input_file_name
    test_i = pd.read_csv('./SVM_Data/TEST/' + input_file_name )
    test_i = test_i.fillna(0)
    
    output_file_name = 'test_' + str(i) + '_dict.txt'
    
    # Rank test_i with given coefficients
    Ai_List = Rank_List(test_i, svm_coefficients_01)
    # Ai_Sorted = sorted(Ai_List.items(), key=operator.itemgetter(1), reverse=True)
    
    writeDict( Ai_List , './SVM_Data/RANK/' + output_file_name, ':')
    # print Rank_List(test_i) 

In [16]:
import operator
for i in range(File_List_Len) :
    input_file_name = 'test_' + str(i) + '_dict.txt'
    output_file_name = 'test_' + str(i) + '_sorted.txt'
    # print input_file_name
    Ai_Dict = readDict('./SVM_Data/RANK/' + input_file_name, ':')
    # print type(Ai_Dict)
    Ai_Sorted = sorted(Ai_Dict.items(), key=operator.itemgetter(1), reverse=True)
    # print Ai_Sorted
    writeList(Ai_Sorted, './SVM_Data/RANK/' + output_file_name)
    # writeDict(Ai_Sorted, './SVM_Data/RANK/' + output_file_name, ':')

In [17]:
def Get_Sum_Score( key ) :
    sum_score = 0.0
    for i in range(File_List_Len) :
        input_file_name = 'test_' + str(i) + '_dict.txt'
        Ai_Dict = readDict('./SVM_Data/RANK/' + input_file_name, ':')
        # print type(Ai_Dict[key])
        # print Ai_Dict[key] 
        # if Ai_Dict[key] is None :
            # Ai_Dict[key] = 0
        if key in Ai_Dict :
            # print Ai_Dict[key]
            sum_score += Ai_Dict[key]
    return sum_score

In [18]:
Get_Sum_Score( 'A_0_score' )

166.0

In [19]:
Ave_List = {}
for i in range(10) :
    
    key = 'A_' + str(i) + '_score'
    
    sum_score = Get_Sum_Score( key )
    ave_score = float(sum_score) / File_List_Len
    
    Ave_List[key] = ave_score

Ave_List = sorted(Ave_List.items(), key=operator.itemgetter(1), reverse=True)
# print ave_score

In [20]:
Ave_List

[('A_0_score', 1.66),
 ('A_1_score', 1.04),
 ('A_9_score', 0.14),
 ('A_6_score', -0.02),
 ('A_4_score', -0.1),
 ('A_7_score', -0.12),
 ('A_8_score', -0.24),
 ('A_5_score', -0.58),
 ('A_3_score', -0.68),
 ('A_2_score', -1.02)]

In [21]:
# len( Ave_List )

In [22]:
# row 代表读取文件的第几行
# rank 代表要匹配这一行的答案

def hit( filename, row, rank ) :
    # print type(rank)
    hit = 0
    with open('./SVM_Data/RANK/' + filename, 'r') as f:
        
        while( int(row) > 0) :
            f.readline()
            row = row - 1
        
        answer = f.readline()
        answer_key = answer.split(':')[0]
        # print answer_key
        answer_rank = answer_key.split('_')[1]
        # print answer_rank
        if int(rank) == int(answer_rank) :
                hit = 1
        else :
            hit = 0
    return hit

In [26]:
model_hit_best = 0
model_hit_second = 0
model_hit_third = 0 
model_hit_4th = 0
model_hit_5th = 0
for i in range(File_List_Len) :
    
    input_file_name = 'test_' + str(i) + '_sorted.txt'
    
    model_hit_best += hit( input_file_name, 0, 0 )
    
    model_hit_second += hit( input_file_name, 0, 1 )
    
    model_hit_third += hit( input_file_name, 0, 2)
    
    model_hit_4th += hit( input_file_name, 0, 3)
    
    model_hit_5th += hit( input_file_name, 0, 4)
    
print model_hit_best, model_hit_second, model_hit_third, model_hit_4th, model_hit_5th
    

33 19 10 11 12
